# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 26 2023 3:14PM,256030,16,TASA-285001,TASA USA Inc.,Released
1,Jan 26 2023 3:10PM,256029,16,TASA-284197,TASA USA Inc.,Released
2,Jan 26 2023 3:05PM,256028,16,TASA-284198,TASA USA Inc.,Released
3,Jan 26 2023 2:54PM,256027,19,9081379,"Emersa Waterbox, LLC",Released
4,Jan 26 2023 2:27PM,256022,10,0086333234,ISDIN Corporation,Released
5,Jan 26 2023 2:27PM,256022,10,0086333235,ISDIN Corporation,Released
6,Jan 26 2023 2:27PM,256022,10,0086333241,ISDIN Corporation,Released
7,Jan 26 2023 2:27PM,256022,10,0086333242,ISDIN Corporation,Released
8,Jan 26 2023 2:27PM,256022,10,0086333375,ISDIN Corporation,Executing
9,Jan 26 2023 2:27PM,256022,10,0086333376,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,256022,Released,5
18,256027,Released,1
19,256028,Released,1
20,256029,Released,1
21,256030,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256022,NaN,1.0,5.0
256027,NaN,NaN,1.0
256028,NaN,NaN,1.0
256029,NaN,NaN,1.0
256030,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255953,42.0,14.0,1.0
255969,0.0,1.0,0.0
255987,0.0,21.0,28.0
255994,0.0,18.0,3.0
255997,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255953,42,14,1
255969,0,1,0
255987,0,21,28
255994,0,18,3
255997,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255953,42,14,1
1,255969,0,1,0
2,255987,0,21,28
3,255994,0,18,3
4,255997,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255953,42,14,1
1,255969,,1,
2,255987,,21,28
3,255994,,18,3
4,255997,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 26 2023 3:14PM,256030,16,TASA USA Inc.
1,Jan 26 2023 3:10PM,256029,16,TASA USA Inc.
2,Jan 26 2023 3:05PM,256028,16,TASA USA Inc.
3,Jan 26 2023 2:54PM,256027,19,"Emersa Waterbox, LLC"
4,Jan 26 2023 2:27PM,256022,10,ISDIN Corporation
10,Jan 26 2023 1:40PM,256017,10,ISDIN Corporation
32,Jan 26 2023 1:29PM,256013,19,"GUSA Granules USA, Inc."
33,Jan 26 2023 1:13PM,256011,15,"Person & Covey, Inc."
37,Jan 26 2023 1:09PM,256009,15,"Virtus Pharmaceuticals, LLC"
38,Jan 26 2023 1:07PM,256007,15,"Mizner Bioscience, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 26 2023 3:14PM,256030,16,TASA USA Inc.,,,1
1,Jan 26 2023 3:10PM,256029,16,TASA USA Inc.,,,1
2,Jan 26 2023 3:05PM,256028,16,TASA USA Inc.,,,1
3,Jan 26 2023 2:54PM,256027,19,"Emersa Waterbox, LLC",,,1
4,Jan 26 2023 2:27PM,256022,10,ISDIN Corporation,,1,5
5,Jan 26 2023 1:40PM,256017,10,ISDIN Corporation,,,22
6,Jan 26 2023 1:29PM,256013,19,"GUSA Granules USA, Inc.",,1,
7,Jan 26 2023 1:13PM,256011,15,"Person & Covey, Inc.",,,4
8,Jan 26 2023 1:09PM,256009,15,"Virtus Pharmaceuticals, LLC",,,1
9,Jan 26 2023 1:07PM,256007,15,"Mizner Bioscience, LLC",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 26 2023 3:14PM,256030,16,TASA USA Inc.,1,,
1,Jan 26 2023 3:10PM,256029,16,TASA USA Inc.,1,,
2,Jan 26 2023 3:05PM,256028,16,TASA USA Inc.,1,,
3,Jan 26 2023 2:54PM,256027,19,"Emersa Waterbox, LLC",1,,
4,Jan 26 2023 2:27PM,256022,10,ISDIN Corporation,5,1,
5,Jan 26 2023 1:40PM,256017,10,ISDIN Corporation,22,,
6,Jan 26 2023 1:29PM,256013,19,"GUSA Granules USA, Inc.",,1,
7,Jan 26 2023 1:13PM,256011,15,"Person & Covey, Inc.",4,,
8,Jan 26 2023 1:09PM,256009,15,"Virtus Pharmaceuticals, LLC",1,,
9,Jan 26 2023 1:07PM,256007,15,"Mizner Bioscience, LLC",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 26 2023 3:14PM,256030,16,TASA USA Inc.,1,,
1,Jan 26 2023 3:10PM,256029,16,TASA USA Inc.,1,,
2,Jan 26 2023 3:05PM,256028,16,TASA USA Inc.,1,,
3,Jan 26 2023 2:54PM,256027,19,"Emersa Waterbox, LLC",1,,
4,Jan 26 2023 2:27PM,256022,10,ISDIN Corporation,5,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 26 2023 3:14PM,256030,16,TASA USA Inc.,1.0,NaN,NaN
1,Jan 26 2023 3:10PM,256029,16,TASA USA Inc.,1.0,NaN,NaN
2,Jan 26 2023 3:05PM,256028,16,TASA USA Inc.,1.0,NaN,NaN
3,Jan 26 2023 2:54PM,256027,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
4,Jan 26 2023 2:27PM,256022,10,ISDIN Corporation,5.0,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 26 2023 3:14PM,256030,16,TASA USA Inc.,1.0,0.0,0.0
1,Jan 26 2023 3:10PM,256029,16,TASA USA Inc.,1.0,0.0,0.0
2,Jan 26 2023 3:05PM,256028,16,TASA USA Inc.,1.0,0.0,0.0
3,Jan 26 2023 2:54PM,256027,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
4,Jan 26 2023 2:27PM,256022,10,ISDIN Corporation,5.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,768033,30.0,19.0,0.0
15,1024014,36.0,21.0,0.0
16,1024093,4.0,0.0,0.0
19,1023990,3.0,15.0,42.0
21,511973,0.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,768033,30.0,19.0,0.0
1,15,1024014,36.0,21.0,0.0
2,16,1024093,4.0,0.0,0.0
3,19,1023990,3.0,15.0,42.0
4,21,511973,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,30.0,19.0,0.0
1,15,36.0,21.0,0.0
2,16,4.0,0.0,0.0
3,19,3.0,15.0,42.0
4,21,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,30.0
1,15,Released,36.0
2,16,Released,4.0
3,19,Released,3.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,0.0,0.0,0.0,42.0,0.0
Executing,19.0,21.0,0.0,15.0,2.0
Released,30.0,36.0,4.0,3.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,0.0,0.0,0.0,42.0,0.0
1,Executing,19.0,21.0,0.0,15.0,2.0
2,Released,30.0,36.0,4.0,3.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,0.0,0.0,0.0,42.0,0.0
1,Executing,19.0,21.0,0.0,15.0,2.0
2,Released,30.0,36.0,4.0,3.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()